In [1]:
######################### Carrier ##############################
########################## Notes #############################
# The 18 equipments' data include 'Bypass Factor' at each row.
# The variable we have is TC(Total capacity), SHC(Sensible Capacity), kW(power) and BF(bypass factor).

# On page 34, they show the method to correct SHC, since The shown SHC in original excel is based on 
# 80F edb temp of air entering coil. 

In [1]:
# !pip3 install metpy
# !pip3 install pathlib # instal if needed
# not work when connect to PNNL VPN

import pandas as pd
import numpy as np
import metpy.calc as mcalc
from metpy.units import units
from pathlib import Path
from bisect import bisect_left


C:\Users\zhan682\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Correction Factor
def calculate_Factor(bf_read, factors):
    bf05, bf10, bf20 = factors[0], factors[1], factors[2]
    if bf_read == 0.05:
        return bf05
    elif bf_read == 0.10:
        return bf10
    elif bf_read == 0.20:
        return bf20
    elif bf_read < 0.05:
        return bf05 - ((0.05 - bf_read) * (bf10 - bf05) / 0.05)
    elif bf_read > 0.05 and bf_read < 0.10:
        return bf05 + ((bf_read - 0.05) * (bf10 - bf05) / 0.05)
    elif bf_read > 0.10 and bf_read < 0.20:
        return bf10 + ((bf_read - 0.10) * (bf20 - bf10) / 0.1)
    else:
        return bf20 + ((bf_read - 0.2) * (bf20 - bf10) / 0.1)

# 48-50A-21PD-SM
def correctionFactor_1(row):
    factors = []
    # get correction factor
    if row['EADB'] in [79, 81]:
        factors = [1.04, 0.98, 0.87]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [78, 82]:
        factors = [2.07, 1.96, 1.74]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [77, 83]:
        factors = [3.11, 2.94, 2.62]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [76, 84]:
        factors = [4.14, 3.92, 3.49]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [75, 85]:
        factors = [4.18, 4.91, 4.36]
        factor = calculate_Factor(row['BF'], factors)
    else:
        factor = 1.1 * (1 - row['BF']) * (row['EADB'] - 80)
    
    modified_row = row.copy()
        
    # modify 'SHC'
    if row['EADB'] < 80:
        modified_row['SHC'] = row['SHC'] - factor * row['airflow']
    elif row['EADB'] > 80:
        modified_row['SHC'] = row['SHC'] + factor * row['airflow']
    return modified_row


In [3]:
# Supply Fan Power
# esp
esp_ranges = [
    (0, 28.8, 0.1),
    (29, 42.5, 0.15),
    (43, 70, 0.2),
    (71, 105, 0.25),
    (106, 134, 0.3),
    (135, 210, 0.35),
    (211, 280, 0.4),
    (281, 350, 0.45),
    (351, 400, 0.55),
    (401, 500, 0.65),
    (501, float('inf'), 0.75)
]

def calculate_esp(TC):
    TC = TC//1000
    for lower, upper, esp in esp_ranges:
        if lower <= TC <= upper:
            return esp


In [4]:
def loadData(name,cols,nHead,nRows,sheet_name):
    ## Load the data from a page in the excel file
    df = pd.read_excel(str(name), sheet_name, header=int(nHead), 
                       usecols=cols, skiprows=None, nrows=int(nRows), engine="openpyxl")
    df = df.replace('—', np.nan)
    
    ## Rotate data
    column_index_to_drop = [0]
    rotate_df = df.drop(df.columns[column_index_to_drop], axis=1)
    buffer_df = rotate_df.T
    buffer_df.reset_index(inplace=True)
    buffer_df.columns = buffer_df.iloc[0]
    buffer_df = buffer_df.iloc[1:]
    
    ## Read EAWB
    eawb_values = buffer_df.iloc[:5, 0]
    eawb_values = eawb_values.tolist()
    eawb_times = len(buffer_df)//5
    eawb_list = eawb_values * eawb_times
    buffer_df.rename(columns={'Unnamed: 1':'EAWB'}, inplace=True)
    buffer_df['EAWB'] = eawb_list

    
    ## Read cfm
    cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3), 
                       usecols=cols, skiprows=None, nrows=1, engine="openpyxl")
    cfm_values = np.array(cfm)[0]
    cfm_values = cfm_values.tolist()
    cfm_list = [int(value) for value in cfm_values if not pd.isna(value)]
    cfm_array = np.repeat(cfm_list,5).tolist()
    buffer_df.insert(0, "airflow", np.nan)
    buffer_df['airflow'] = cfm_array
    
    ## Modify the dataframe
    num_chunk = len(buffer_df.columns)//4
    if num_chunk <= 5:
        chunk1 = buffer_df.iloc[:, :2]
        chunk2 = buffer_df.iloc[:, 2:6]
        chunk3 = buffer_df.iloc[:, 6:10]
        chunk4 = buffer_df.iloc[:, 10:14]
        chunk5 = buffer_df.iloc[:, 14:18]
        chunk6 = buffer_df.iloc[:, 18:22]
        result_df = pd.concat([chunk2, chunk3, chunk4, chunk5, chunk6], ignore_index=True)
        chunk1_df = pd.concat([chunk1, chunk1, chunk1, chunk1, chunk1], ignore_index=True)
        chunk1_df.reset_index(inplace=True)
    else:
        chunk1 = buffer_df.iloc[:, :2]
        chunk2 = buffer_df.iloc[:, 2:6]
        chunk3 = buffer_df.iloc[:, 6:10]
        chunk4 = buffer_df.iloc[:, 10:14]
        chunk5 = buffer_df.iloc[:, 14:18]
        chunk6 = buffer_df.iloc[:, 18:22]
        chunk7 = buffer_df.iloc[:, 22:26]
        result_df = pd.concat([chunk2, chunk3, chunk4, chunk5, chunk6, chunk7], ignore_index=True)
        chunk1_df = pd.concat([chunk1, chunk1, chunk1, chunk1, chunk1, chunk1], ignore_index=True)
        chunk1_df.reset_index(inplace=True)   
    merged_df = pd.concat([chunk1_df, result_df], axis=1)
    merged_df = merged_df.iloc[:, 1:]
    
    ## Read EADB
    eadb_values = df.iloc[:, 0].tolist()
    eadb_values = [int(value) for value in eadb_values if not pd.isna(value)]
    eadb_times = 5 * len(cfm_list)
    eadb_array = np.repeat(eadb_values,eadb_times).tolist()
    merged_df['EADB'] = 80 # Standard Mode
    merged_df['OADB'] = eadb_array
    
    ## Calculate RH
    # The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
    # And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
    pressure = 101325 * units.pascal
    # Convert Celsius to Kelvin and compute relative humidity
    merged_df['RH'] = merged_df.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['EADB'] * units.degF, row['EAWB'] * units.degF).to('percent').magnitude, axis=1)
    
    ## Modify units
    merged_df['TC'] = merged_df['TC']*1000
    merged_df['SHC'] = merged_df['SHC']*1000
#     ## Correct SHC
#     merged_df = merged_df.apply(lambda row: correctionFactor_1(row), axis=1)
    
    ## Clean NA
    merged_df = merged_df.dropna()
    merged_df = merged_df.reset_index(drop=True)
    
    ## Supply fan power
    merged_df.insert(0, "SFP", np.nan)
    merged_df['TC'] = pd.to_numeric(merged_df['TC'], errors='coerce')
    merged_df['ESP'] = merged_df['TC'].apply(calculate_esp)
    ## organize
#     # for test
#     merged_df['S2T'] = merged_df['SHC']/merged_df['TC']
#     new_cols = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'S2T', 'kW', 'airflow']

    new_cols = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'kW', 'airflow', 'ESP', 'SFP', 'BF']
    final = merged_df[new_cols]
    
    return final

In [5]:
# 48-50A-21PD-FP
# excel function can do this
bhp_ranges_21_20 = [
    (4000, 2.21, 2.38, 2.55, 2.71),
    (5000, 2.71, 2.89, 3.08, 3.26),
    (6000, 3.29, 3.5, 3.7, 3.9),
    (7000, 3.96, 4.19, 4.41, 4.63),
    (7500, 4.33, 4.56, 4.79, 5.02),
    (8000, 4.73, 4.97, 5.21, 5.45),
    (9000, 5.6, 5.85, 6.11, 6.37),
    (10000, 6.58, 6.85, 7.13, 7.4)
]

bhp_ranges_21_30 = [
    (5000, 0.85, 1.03, 1.22, 1.42),
    (6000, 1.26, 1.45, 1.65, 1.86),
    (7000, 1.79, 1.98, 2.19, 2.41),
    (8000, 2.44, 2.64, 2.85, 3.08),
    (9000, 3.23, 3.43, 3.65, 3.88),
    (10000, 4.15, 4.36, 4.58, 4.82),
    (11000, 5.22, 5.44, 5.67, 5.91),
    (12000, 6.45, 6.67, 6.90, 7.15),
    (13000, 7.85, 8.07, 8.30, 8.55),
    (14000, 9.41, 9.64, 9.88, 10.13),
    (15000, 11.15, 11.38, 11.62, 11.88)
]

bhp_ranges_21_35 = [
    (7000, 1.96, 2.22, 2.47, 2.72),
    (8000, 2.62, 2.89, 3.16, 3.43),
    (9000, 3.39, 3.68, 3.97, 4.26),
    (10000, 4.29, 4.61, 4.91, 5.22),
    (10500, 4.80, 5.12, 5.43, 5.75),
    (11000, 5.33, 5.66, 5.99, 6.31),
    (12000, 6.52, 6.86, 7.21, 7.54),
    (13000, 7.85, 8.21, 8.57, 8.92),
    (14000, 9.33, 9.71, 10.09, 10.45),
    (15000, 10.98, 11.37, 11.76, 12.15),
    (16000, 12.79, 13.2, 13.6, 14),
    (17000, 14.77, 15.19, 15.61, 16.03),
    (17500, 15.83, 16.25, 16.68, 17.1)
]

bhp_ranges_21_40 = [
    (8000, 2.69, 3.08, 3.47, 3.86),
    (9000, 3.53, 3.94, 4.36, 4.78),
    (10000, 4.52, 4.96, 5.4, 5.84),
    (11000, 5.68, 6.14, 6.6, 7.07),
    (12000, 7.01, 7.49, 7.98, 8.47),
    (13000, 8.53, 9.03, 9.54, 10.05),
    (14000, 10.24, 10.76, 11.29, 11.82),
    (15000, 12.15, 12.69, 13.24, 13.79),
    (16000, 14.27, 14.83, 15.4, 15.97),
    (17000, 16.61, 17.19, 17.77, 18.36),
    (18000, 19.18, 19.77, 20.37, 20.98),
    (19000, 21.98, 22.59, 23.21, 23.84),
    (20000, 25.02, 25.65, 26.29, 26.93)
]

bhp_ranges_21_50 = [
    (10000, 4.62, 5.06, 5.5, 5.95),
    (11000, 5.8, 6.26, 6.73, 7.2),
    (12000, 7.15, 7.63, 8.12, 8.62),
    (13000, 8.69, 9.19, 9.7, 10.22),
    (14000, 10.43, 10.95, 11.48, 12.01),
    (15000, 12.37, 12.91, 13.46, 14.01),
    (16000, 14.52, 15.08, 15.65, 16.22),
    (17000, 16.89, 17.48, 18.06, 18.65),
    (18000, 19.5, 20.1, 20.71, 21.32),
    (19000, 22.35, 22.96, 23.59, 24.21),
    (20000, 25.43, 26.07, 26.71, 27.36)
]

bhp_ranges_21_60 = [
    (12000, 4.02, 4.71, 5.41, 6.12),
    (14000, 5.74, 6.49, 7.24, 8.01),
    (15000, 6.75, 7.53, 8.32, 9.11),
    (16000, 7.88, 8.68, 9.5, 10.32),
    (17000, 9.12, 9.95, 10.79, 11.64),
    (18000, 10.48, 11.33, 12.2, 13.07),
    (19000, 11.96, 12.84, 13.74, 14.64),
    (20000, 13.57, 14.47, 15.4, 16.32),
    (21000, 15.3, 16.24, 17.19, 18.14),
    (22000, 17.18, 18.14, 19.11, 20.09),
    (23000, 19.2, 20.18, 21.17, 22.17),
    (24000, 21.35, 22.36, 23.38, 24.4),
    (25000, 23.66, 24.68, 25.72, 26.78),
    (26000, 26.11, 27.16, 28.23, 29.3),
    (27000, 28.72, 29.79, 30.88, 31.97)
]

# 48-50P-12PD-FP
bhp_ranges_12_30 = [
    (6000, 0.59, 0.91, 1.27, 1.66),
    (7500, 0.94, 1.28, 1.68, 2.11),
    (9000, 1.46, 1.8, 2.22, 2.69),
    (10500, 2.16, 2.52, 2.95, 3.43),
    (12000, 3.08, 3.44, 3.89, 4.39),
    (13500, 4.25, 4.62, 5.07, 5.58),
    (15000, 5.69, 6.06, 6.52, 7.04)
]

bhp_ranges_12_35 = [
    (7000, 246, 301, 1.58, 398),
    (8000, 266, 315, 1.92, 406),
    (10000, 310, 350, 2.8, 427),
    (12000, 357, 390, 4.06, 457),
    (14000, 406, 435, 5.76, 492),
    (15000, 430, 458, 6.8, 511)
]

bhp_ranges_12_40 = [
    (8000, 0.98, 1.33, 1.72, 2.14),
    (10000, 1.67, 2.11, 2.55, 3.01),
    (12000, 2.65, 3.18, 3.7, 4.23),
    (14000, 3.96, 4.61, 5.22, 5.83),
    (16000, 5.67, 6.44, 7.15, 7.85),
    (18000, 7.84, 8.72, 9.55, 10.34),
    (20000, 10.51, 11.51, 12.46, 13.36)
]

bhp_ranges_12_50 = [
    (9000, 1.34, 1.73, 2.15, 2.6),
    (10000, 1.74, 2.17, 2.62, 3.09),
    (12000, 2.76, 3.29, 3.81, 4.35),
    (14000, 4.15, 4.79, 5.4, 6.01),
    (16000, 5.96, 6.71, 7.42, 8.11),
    (18000, 8.26, 9.12, 9.93, 10.72),
    (20000, 11.1, 12.06, 12.99, 13.88)
]

bhp_ranges_12_55 = [
    (10000, 1.04, 1.5, 2.03, 2.66),
    (12500, 1.69, 2.23, 2.78, 3.4),
    (15000, 2.59, 3.23, 3.85, 4.51),
    (17500, 3.78, 4.52, 5.24, 5.97),
    (20000, 5.31, 6.15, 6.98, 7.79),
    (22500, 7.23, 8.16, 9.09, 10),
    (25000, 9.59, 10.6, 11.62, 12.64)
]

bhp_ranges_12_60 = [
    (12000, 1.54, 2.03, 2.57, 3.2),
    (15000, 2.65, 3.27, 3.88, 4.53),
    (18000, 4.22, 5, 5.73, 6.46),
    (21000, 6.36, 7.29, 8.18, 9.04),
    (24000, 9.19, 10.24, 11.29, 12.29),
    (27000, 12.8, 13.93, 15.14, 16.3),
    (30000, 17.29, 18.5, 19.82, 21.15)
]

bhp_ranges_12_70 = [
    (14000, 2.23, 2.8, 3.38, 4.02),
    (17500, 3.92, 4.67, 5.39, 6.1),
    (21000, 6.36, 7.29, 8.18, 9.04),
    (24500, 9.74, 10.8, 11.88, 12.91),
    (28000, 14.18, 15.35, 16.6, 17.82),
    (30000, 17.29, 18.5, 19.82, 21.15)
]

bhp_ranges_12_75 = [
    (15000, 3.02, 3.91, 4.83, 5.8),
    (16000, 3.52, 4.45, 5.41, 6.42),
    (18000, 4.68, 5.71, 6.76, 7.84),
    (20000, 6.08, 7.22, 8.36, 9.52),
    (22000, 7.75, 9, 10.24, 11.49),
    (24000, 9.71, 11.07, 12.41, 13.75),
    (26000, 11.99, 13.46, 14.9, 16.34),
    (28000, 14.61, 16.18, 17.73, 19.28),
    (30000, 17.6, 19.28, 20.93, 22.57)
]

bhp_ranges_12_90 = [
    (16000, 3.27, 4.06, 4.88, 5.74),
    (18000, 4.42, 5.31, 6.21, 7.14),
    (20000, 5.83, 6.82, 7.81, 8.82),
    (22000, 7.53, 8.62, 9.7, 10.8),
    (24000, 9.55, 10.75, 11.93, 13.11),
    (26000, 11.91, 13.22, 14.5, 15.78),
    (28000, 14.65, 16.06, 17.45, 18.83),
    (30000, 17.78, 19.31, 20.8, 22.28),
    (32000, 21.36, 22.99, 24.59, 26.17),
    (34000, 25.39, 27.13, 28.84, 30.53)
]

bhp_ranges_12_100 = [
    (20000, 5.83, 6.82, 7.81, 8.82),
    (22000, 7.53, 8.62, 9.7, 10.8),
    (24000, 9.55, 10.75, 11.93, 13.11),
    (26000, 11.91, 13.22, 14.5, 15.78),
    (28000, 14.65, 16.06, 17.45, 18.83),
    (30000, 17.78, 19.31, 20.8, 22.28),
    (32000, 21.36, 22.99, 24.59, 26.17),
    (34000, 25.39, 27.13, 28.84, 30.53),
    (36000, 29.92, 31.77, 33.58, 35.38),
    (38000, 34.96, 36.91, 38.85, 40.74),
    (40000, 40.54, 42.61, 44.65, 46.66)
]

def get_scaled_tuple(airflow, bhp_ranges):
    airflow = int(airflow)
    cfm_values = [t[0] for t in bhp_ranges]
    idx = bisect_left(cfm_values, airflow)
    
    if idx == 0:
        return bhp_ranges[0]
    elif idx == len(bhp_ranges):
        return bhp_ranges[-1]
    
    lower_tuple = bhp_ranges[idx - 1]
    upper_tuple = bhp_ranges[idx]
    
    scale_factor = (airflow - lower_tuple[0]) / (upper_tuple[0] - lower_tuple[0])
    
    scaled_values = [round((1 - scale_factor) * l + scale_factor * u, 2) for l, u in zip(lower_tuple[1:], upper_tuple[1:])]
    
    scaled_tuple = (airflow, *scaled_values)
    return scaled_tuple

def find_matching_tuple(airflow, bhp_ranges):
    for tpl in bhp_ranges:
        if tpl[0] == airflow:
            return tpl
    return get_scaled_tuple(airflow, bhp_ranges)

def calculate_custom_bhp(cfm, esp, bhp_ranges):
    result_tuple = find_matching_tuple(cfm, bhp_ranges)
    esp02, esp04, esp06, esp08 = result_tuple[1], result_tuple[2], result_tuple[3], result_tuple[4]
    
    if esp == 0.2:
        return esp02
    elif esp == 0.4:
        return esp04
    elif esp == 0.6:
        return esp06
    elif esp >= 0.8:
        return esp08
    elif esp < 0.2:
        return esp02 - ((0.2 - esp) * (esp04 - esp02) / 0.2)
    elif esp > 0.2 and esp < 0.4:
        return esp02 + ((esp - 0.2) * (esp04 - esp02) / 0.2)
    elif esp > 0.4 and esp < 0.6:
        return esp04 + ((esp - 0.4) * (esp06 - esp04) / 0.2)
    elif esp > 0.6 and esp < 0.8:
        return esp06 + ((esp - 0.6) * (esp08 - esp06) / 0.2)

def save_sfp(df,bhp_ranges,eff):
    df['BHP'] = df.apply(lambda row: calculate_custom_bhp(row['airflow'], row['ESP'], bhp_ranges), axis=1)
    df['SFP'] = df['BHP']*746/eff
    new_cols = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'kW', 'airflow', 'SFP', 'BF']
    final = df[new_cols]
    return final

In [12]:
# 48-50A-21PD-SubCool

# column_names = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'kW', 'airflow', 'ESP', 'SFP']
# gap = pd.DataFrame(columns=column_names)
# new_row = pd.Series(['-------------'] * len(column_names), index=column_names)
# gap = gap.append(new_row, ignore_index=True)

# 020
test020_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 1")
test020_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 1")
test020 = pd.concat([test020_1,test020_2], axis=0)
test020 = save_sfp(test020, bhp_ranges_21_20, 0.895)

# 025
test025_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 2")
test025_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 2")
test025 = pd.concat([test025_1,test025_2], axis=0)
test025 = save_sfp(test025, bhp_ranges_21_30, 0.895)

# 027
test027_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 3")
test027_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 3")
test027 = pd.concat([test027_1,test027_2], axis=0)
test027 = save_sfp(test027, bhp_ranges_21_30, 0.895)

# 030
test030_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 4")
test030_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 4")
test030 = pd.concat([test030_1,test030_2], axis=0)
test030 = save_sfp(test030, bhp_ranges_21_30, 0.895)

# 035
test035_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 5")
test035_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 5")
test035 = pd.concat([test035_1,test035_2], axis=0)
test035 = save_sfp(test035, bhp_ranges_21_35, 0.895)

# 040
test040_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 6")
test040_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 6")
test040 = pd.concat([test040_1,test040_2], axis=0)
test040 = save_sfp(test040, bhp_ranges_21_40, 0.895)

# 050
test050_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 7")
test050_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 7")
test050 = pd.concat([test050_1,test050_2], axis=0)
test050 = save_sfp(test050, bhp_ranges_21_50, 0.895)

# 060
test060_1 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:V",8,20, "page 8")
test060_2 = loadData('./excel/48-50A-21PD-SubCool.xlsx',"A:Q",34,20, "page 8")
test060 = pd.concat([test060_1,test060_2], axis=0)
test060 = save_sfp(test060, bhp_ranges_21_60, 0.895)


final = pd.concat([test020,test025,test027,test030, test035, test040, test050, test060], axis=0)
file_path = Path('./output/ready/48-50A-21PD-SubCool.csv')
final.to_csv(file_path, index=False)


C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name

In [14]:
# 48-50P-12PD-SubCool

# 030
# Standard Capacity Coil
samp030_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 1")
samp030_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",34,20, "page 1")
# High-Capacity Coil
samp030_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 2")
samp030_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",34,20, "page 2")

samp030 = pd.concat([samp030_1,samp030_2,samp030_3,samp030_4], axis=0)
samp030 = save_sfp(samp030, bhp_ranges_12_30, 0.895)

# 035 Standard Capacity Coil
samp035_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 3")
samp035_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",34,20, "page 3")
samp035 = pd.concat([samp035_1,samp035_2], axis=0)
samp035 = save_sfp(samp035, bhp_ranges_12_35, 0.895)

# 040 
# Standard Capacity Coil
samp040_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 4")
samp040_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",34,20, "page 4")
# High-Capacity Coil
samp040_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 5")
samp040_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",34,20, "page 5")

samp040 = pd.concat([samp040_1,samp040_2,samp040_3,samp040_4], axis=0)
samp040 = save_sfp(samp040, bhp_ranges_12_40, 0.895)


# 050
# Standard Capacity Coil
samp050_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 6")
samp050_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 6")
# High-Capacity Coil
samp050_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 7")
samp050_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 7")

samp050 = pd.concat([samp050_1,samp050_2,samp050_3,samp050_4], axis=0)
samp050 = save_sfp(samp050, bhp_ranges_12_50, 0.895)
                     
# 055
# Standard Capacity Coil
samp055_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 8")
samp055_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 8")
# High-Capacity Coil
samp055_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 9")
samp055_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 9")

samp055 = pd.concat([samp055_1,samp055_2,samp055_3,samp055_4], axis=0)
samp055 = save_sfp(samp055, bhp_ranges_12_55, 0.895)

# 060
# Standard Capacity Coil
samp060_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 10")
samp060_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 10")
# High-Capacity Coil
samp060_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 11")
samp060_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 11")

samp060 = pd.concat([samp060_1,samp060_2,samp060_3,samp060_4], axis=0)
samp060 = save_sfp(samp060, bhp_ranges_12_60, 0.895)

# 070
# Standard Capacity Coil
samp070_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 12")
samp070_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 12")
# High-Capacity Coil
samp070_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 13")
samp070_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 13")

samp070 = pd.concat([samp070_1,samp070_2,samp070_3,samp070_4], axis=0)
samp070 = save_sfp(samp070, bhp_ranges_12_70, 0.895)

# 075
# Standard Capacity Coil
samp075_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 14")
samp075_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 14")
# High-Capacity Coil
samp075_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 15")
samp075_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 15")

samp075 = pd.concat([samp075_1,samp075_2,samp075_3,samp075_4], axis=0)
samp075 = save_sfp(samp075, bhp_ranges_12_75, 0.895)

# 090
# Standard Capacity Coil
samp090_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 16")
samp090_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 16")
# High-Capacity Coil
samp090_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 17")
samp090_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 17")
samp090 = pd.concat([samp090_1,samp090_2,samp090_3,samp090_4], axis=0)
samp090 = save_sfp(samp090, bhp_ranges_12_90, 0.895)

# 100
# Standard Capacity Coil
samp100_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",8,20, "page 18")
samp100_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 18")
# High-Capacity Coil
samp100_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",8,20, "page 19")
samp100_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 19")
samp100 = pd.concat([samp100_1,samp100_2,samp100_3,samp100_4], axis=0)
samp100 = save_sfp(samp100, bhp_ranges_12_100, 0.895)


final2 = pd.concat([samp030, samp035, samp040, samp050, samp055, samp060, samp070, samp075, samp090, samp100], axis=0)
file2_path = Path('./output/ready/48-50P-12PD-SubCool.csv')
final2.to_csv(file2_path, index=False)

C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_12012\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name

In [7]:
# 050
# # Standard Capacity Coil
# samp050_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 6")
# samp050_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 6")
# High-Capacity Coil
samp050_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 7")
samp050_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 7")

# samp050 = pd.concat([samp050_1,samp050_2,samp050_3,samp050_4], axis=0)
samp050_h = pd.concat([samp050_3,samp050_4], axis=0)
samp050_h = save_sfp(samp050_h, bhp_ranges_12_50, 0.895)
samp050_h.to_csv('./output/12PD-SubCool-50_High.csv', index=False)
                     
# 070
# Standard Capacity Coil
samp070_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 12")
samp070_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 12")
# High-Capacity Coil
samp070_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 13")
samp070_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 13")

samp070 = pd.concat([samp070_1,samp070_2,samp070_3,samp070_4], axis=0)
samp070 = save_sfp(samp070, bhp_ranges_12_70, 0.895)

# 075
# Standard Capacity Coil
samp075_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 14")
samp075_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 14")
# High-Capacity Coil
samp075_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 15")
samp075_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 15")

samp075 = pd.concat([samp075_1,samp075_2,samp075_3,samp075_4], axis=0)
samp075 = save_sfp(samp075, bhp_ranges_12_75, 0.895)

# 090
# Standard Capacity Coil
samp090_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 16")
samp090_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 16")
# High-Capacity Coil
samp090_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:V",8,20, "page 17")
samp090_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:G",34,20, "page 17")
samp090 = pd.concat([samp090_1,samp090_2,samp090_3,samp090_4], axis=0)
samp090 = save_sfp(samp090, bhp_ranges_12_90, 0.895)

# 100
# Standard Capacity Coil
samp100_1 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",8,20, "page 18")
samp100_2 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 18")
# High-Capacity Coil
samp100_3 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:Q",8,20, "page 19")
samp100_4 = loadData('./excel/48-50P-12PD-SubCool.xlsx',"A:L",34,20, "page 19")
samp100 = pd.concat([samp100_1,samp100_2,samp100_3,samp100_4], axis=0)
samp100 = save_sfp(samp100, bhp_ranges_12_100, 0.895)


final2 = pd.concat([samp070, samp075, samp090, samp100], axis=0)
file2_path = Path('./output/12PD-SubCool-after_70.csv')
final2.to_csv(file2_path, index=False)

C:\Users\zhan682\AppData\Local\Temp\ipykernel_20548\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_20548\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_20548\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name, header=int(nHead-3),
C:\Users\zhan682\AppData\Local\Temp\ipykernel_20548\1963626013.py:25: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name